In [ ]:
#AI Instructions

# Investment Strategy:
# Buy-on-Dip: buying into the market only when there is a drop in price.

# Methodology:
# Simulate a buy-on-dip strategy that includes purchasing 1 share of an ETF or stock per 1%, 2%, 3%, 4%, 5% drops in price.

# Monday: Assume XLG closes at $50/share on Monday at closing
# On Monday night, I would enter limit orders for:
# Buy 1 share at $49.50 (99% of the previous days close)
# Buy 1 share at $49.00 (98% of the previous days close)
# Buy 1 share at $48.50 (97% of the previous days close)
# Buy 1 share at $48.00 (96% of the previous days close)
# Buy 1 share at $47.50 (95% of the previous days close)

# Tuesday: Assume XLG drops 3% to 48.50.  
# I would have executed orders at:
# Buy 1 share at $48.50, now owning that 1 share at a purchase price of $48.50
# Buy 1 share at $48.00, now owning that 1 share at a purchase price of $48.00
# Buy 1 share at $47.50, now owning that 1 share at a purchase price of $47.50


In [ ]:
import os
import yfinance as yf 
import pandas as pd
import openpyxl
import datetime as dt

# Ensure /data folder exists
output_folder = "data"
os.makedirs(output_folder, exist_ok=True)

etf_list = [
    "ALLY",
    "FBCG",
    "FMAG",
    "GGLL",
    "GOOGL",
    "HSBC",
    "MGK",
    "MSFT",
    "MSFU",
    "OEF",
    "QLD",
    "QQQ",
    "QQQM",
    "QQUP",
    "QQXL",
    "QTOP",
    "SPLG",
    "SPY",
    "SSO",
    "TQQQ",
    "TOPT",
    "UPRO",
    "VGT",
    "XLG"
]

for ticker_symbol in etf_list:
    ticker = yf.Ticker(ticker_symbol)
    data = ticker.history(period="10y", interval="1d")
    df = pd.DataFrame(data)
    df.drop(['Dividends', 'Stock Splits', 'Volume'], axis=1, inplace=True)
    df.insert(0, 'Date_add', df.index)
    df.insert(1, 'Symbol', ticker_symbol)
    df['week_of_year'] = df.index.isocalendar().week
    df['week_day'] = df.index.day_name()
    df['avg_daily_price'] = df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
    df['Date_add'] = pd.to_datetime(df['Date_add']).dt.strftime('%Y-%m-%d')
    csv_name = os.path.join(output_folder, f'{ticker_symbol.lower()}.csv')
    df.to_csv(csv_name, index=False)

In [ ]:
#AI Instructions

# Investment Strategy:
# Buy-on-Dip: buying into the market only when there is a drop in price.

# Methodology:
# Simulate a buy-on-dip strategy that includes purchasing 1 share of an ETF or stock per 1%, 2%, 3%, 4%, 5% drops in price.

# Monday: Assume XLG closes at $50/share on Monday at closing
# On Monday night, I would enter limit orders for:
# Buy 1 share at $49.50 (99% of the previous days close)
# Buy 1 share at $49.00 (98% of the previous days close)
# Buy 1 share at $48.50 (97% of the previous days close)
# Buy 1 share at $48.00 (96% of the previous days close)
# Buy 1 share at $47.50 (95% of the previous days close)

# Tuesday: Assume XLG drops 3% to 48.50.  
# I would have executed orders at:
# Buy 1 share at $48.50, now owning that 1 share at a purchase price of $48.50
# Buy 1 share at $48.00, now owning that 1 share at a purchase price of $48.00
# Buy 1 share at $47.50, now owning that 1 share at a purchase price of $47.50


#  Partial Legacy Code: Implement buy-on-dip strategy.


purchased_list = []

# Define the buy-on-dip levels
dip_levels = [0.99, 0.98, 0.97, 0.96, 0.95]

# Analyze each row for buy-on-dip opportunities
for index, row in df.iterrows():
    open_price = row['Open']
    low_price = row['Low']
    for level in dip_levels:
        target_price = open_price * level
        if low_price <= target_price:
            # Create a new record to be added
            new_record = {
                'Date': row['Date_add'],
                'Symbol': row['Symbol'],
                'Buy_Price': target_price,
                'Buy_Level': f"{(1 - level) * 100}%"
            }
            purchased_list.append(new_record)

# Create DataFrame from the list of dictionaries
purchased = pd.DataFrame(purchased_list)

# Save the purchased DataFrame to a CSV file
purchased.to_csv(f'{ticker}_bod_purchased.csv', index=False)


In [ ]:
# Load historical data
df = pd.read_csv('yf_{ticker}.csv', parse_dates=['Date_add'])

# Load purchase history
purchased = pd.read_csv('yf_{ticker}_purchased.csv', parse_dates=['Date'])

# Rename columns for clarity
df.rename(columns={'Date_add': 'Date'}, inplace=True)

# Initialize variables to track the cumulative amount invested and shares owned
cumulative_invested = 0
shares_owned = 0

# Initialize lists to track the cumulative invested amount and the market value
cumulative_invested_list = []
market_value_list = []
dates = []

# Track investment values
for index, row in df.iterrows():
    date = row['Date']
    close_price = row['Close']
    
    # Check if any shares were purchased on this date
    daily_purchases = purchased[purchased['Date'] == date]
    for _, purchase in daily_purchases.iterrows():
        shares_owned += 1
        cumulative_invested += purchase['Buy_Price']
    
    # Calculate the market value of the shares owned on this date
    market_value = shares_owned * close_price
    
    # Append values to the lists
    cumulative_invested_list.append(cumulative_invested)
    market_value_list.append(market_value)
    dates.append(date)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Date': dates,
    'Cumulative Invested': cumulative_invested_list,
    'Market Value': market_value_list
})

# Plot the cumulative invested amount and the market value
plt.figure(figsize=(12, 6))
plt.plot(comparison_df['Date'], comparison_df['Cumulative Invested'], label='Cumulative Invested', color='blue')
plt.plot(comparison_df['Date'], comparison_df['Market Value'], label='Market Value (SPLG Close)', color='green')
plt.title('Comparison of Cumulative Invested Amount and Market Value Over Time')
plt.xlabel('Date')
plt.ylabel('Value ($)')
plt.legend()
plt.grid(True)
plt.show()


---

Compare the cumulative dollars invested in XLG against the cumulative value.

In [ ]:
# Load historical data
df = pd.read_csv('yf_xlg.csv', parse_dates=['Date_add'])

# Load purchase history
purchased = pd.read_csv('yf_xlg_purchased.csv', parse_dates=['Date'])

# Rename columns for clarity
df.rename(columns={'Date_add': 'Date'}, inplace=True)

# Initialize variables to track the cumulative amount invested and shares owned
cumulative_invested = 0
shares_owned = 0

# Initialize lists to track the cumulative invested amount and the market value
cumulative_invested_list = []
market_value_list = []
dates = []

# Track investment values
for index, row in df.iterrows():
    date = row['Date']
    close_price = row['Close']
    
    # Check if any shares were purchased on this date
    daily_purchases = purchased[purchased['Date'] == date]
    for _, purchase in daily_purchases.iterrows():
        shares_owned += 1
        cumulative_invested += purchase['Buy_Price']
    
    # Calculate the market value of the shares owned on this date
    market_value = shares_owned * close_price
    
    # Append values to the lists
    cumulative_invested_list.append(cumulative_invested)
    market_value_list.append(market_value)
    dates.append(date)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Date': dates,
    'Cumulative Invested': cumulative_invested_list,
    'Market Value': market_value_list
})

# Plot the cumulative invested amount and the market value
plt.figure(figsize=(12, 6))
plt.plot(comparison_df['Date'], comparison_df['Cumulative Invested'], label='Cumulative Invested', color='blue')
plt.plot(comparison_df['Date'], comparison_df['Market Value'], label='Market Value (XLG Close)', color='green')
plt.title('Comparison of Cumulative Invested Amount and Market Value Over Time')
plt.xlabel('Date')
plt.ylabel('Value ($)')
plt.legend()
plt.grid(True)
plt.show()
